In [1]:
import pandas as pd
import numpy as np
# from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer

stopwords_list = stopwords.words("english")

# from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import recall_score, accuracy_score, f1_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
pos = pd.read_csv('pos.csv')
neg = pd.read_csv('neg.csv')
neg = neg.rename(columns={'0': 'text'})
pos = pos.rename(columns={'0': 'text'})
df = pd.concat([pos, neg])

### Характеристики датасета

In [19]:
doc_lenghts = df['text'].str.split().apply(lambda x: len(x))
doc_count = df.shape[0]

print(f'Количество слов в корпусе {word_count}')
print(f'Количество документов {doc_count}')
print(f'Медианное количестов слов в документе {np.median(doc_lenghts)}')

Количество слов в корпусе 224067
Количество документов 10662
Среднее количестов слов в документе 21
Среднее количестов слов в документе 20.0


In [61]:
# re_tokenizer = RegexpTokenizer(r'[A-z-]+')
# wnl = WordNetLemmatizer()
# df['text'] = df.apply(lambda sent: re_tokenizer.tokenize(sent['text']), axis=1)
# df['text'] = df.apply(lambda sent: [token.lower() for token in sent['text'] if len(token) > 2 and token.isalpha()], axis=1)
# df['text'] = df.apply(lambda sent: [wnl.lemmatize(word) for word in sent['text']], axis=1)
# df['text'] = df.apply(lambda sent: [word for word in sent['text'] if word not in stopwords_list], axis=1)

In [69]:
# vectorizer = CountVectorizer(stop_words=stopwords_list, token_pattern=r'[A-z-]+', ngram_range=(1, 2))
# vectorized = vectorizer.fit_transform(df['text'])
# ngrams_bag = pd.DataFrame(vectorized.toarray())

# char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 8))
# char_vectorized = char_vectorizer.fit_transform(df['text'])
# char_ngrams_bag = pd.DataFrame(char_vectorized.toarray())

In [ ]:
# bw_vectorizer = CountVectorizer(stop_words=stopwords_list, token_pattern=r'[A-z-]+')
# bw_vectorized = bw_vectorizer.fit_transform(df['text'])
# words_bag = pd.DataFrame(bw_vectorized.toarray())
#
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stopwords_list)
# tfidf_vectorized = tfidf_vectorizer.fit_transform(df['text'])
# tfidf_words_bag = pd.DataFrame(tfidf_vectorized.toarray())

In [6]:
xtrain, xvalid, ytrain, yvalid = train_test_split(df['text'], df['sentiment'], test_size=0.25, random_state=17)

pipelines = [
    Pipeline([('ngrams_vectorizer', CountVectorizer(stop_words=stopwords_list, token_pattern=r'[A-z-]+', ngram_range=(1, 2))),
              ('bayes', MultinomialNB())]),
    Pipeline([('char_ngrams_v', CountVectorizer(analyzer='char', ngram_range=(3, 8))),
              ('bayes', MultinomialNB())]),
    Pipeline([('tfidf', CountVectorizer(stop_words=stopwords_list, token_pattern=r'[A-z-]+')),
              ('bag_words_v', MultinomialNB())]),
    Pipeline([('tfidf_v', TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stopwords_list)),
              ('bayes', MultinomialNB())]),
    Pipeline([('tfidf_v', TfidfVectorizer(stop_words=stopwords_list)),
              ('bayes', MultinomialNB())])
]

score = pd.DataFrame(columns=['accuracy', 'precision', 'f1_score', 'recall'])

def get_pred_score(pipe,
                   xtrain=xtrain, ytrain=ytrain,
                   xvalid=xvalid, yvalid=yvalid):
    pipe.fit(xtrain, ytrain)
    predictions = pipe.predict(xvalid)
    row = pd.Series({'accuracy': accuracy_score(yvalid, predictions),
                     'precision': precision_score(yvalid, predictions),
                     'f1_score': f1_score(yvalid, predictions),
                     'recall': recall_score(yvalid, predictions)},
                    name=pipe.steps[0][0])

    return row

for pipe in pipelines:
    score = score.append(get_pred_score(pipe))

In [7]:
score

,accuracy,precision,f1_score,recall
ngrams_v,0.763316,0.763827,0.764112,0.764398
char_ngrams_v,0.776444,0.764454,0.782323,0.801047
tfidf,0.768942,0.772075,0.768595,0.765146
tfidf_v,0.765566,0.773846,0.762988,0.752431
tfidf_v,0.768942,0.768030,0.770321,0.772625
